In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
sys.path.insert(1, str(Path.cwd().parent))
str(Path.cwd().parent)

'c:\\Users\\jaesc2\\GitHub\\skforecast'

In [2]:
import platform
import psutil
import skforecast
import pandas as pd
import numpy as np
import scipy
import sklearn
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from lightgbm import LGBMRegressor
from skforecast.recursive import ForecasterRecursiveMultiSeries
from skforecast.model_selection import grid_search_forecaster_multiseries
from skforecast.utils.utils import align_series_and_exog_multiseries
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import make_column_transformer
from skforecast.preprocessing import series_long_to_dict
from skforecast.preprocessing import exog_long_to_dict
from skforecast.datasets import fetch_dataset
from skforecast.utils import check_preprocess_series


%load_ext pyinstrument
%load_ext line_profiler

In [80]:
n = 5000
n_series = 1000
series = pd.DataFrame(
    {f"series_{i}": np.random.randint(1, 100, n) for i in range(1, n_series)
    },
    index=pd.date_range(start="2023-01-01", periods=n, freq="D"),
)

freq = series.index.freq
series.index.name = "datetime"
series = series.reset_index()
series = pd.melt(series, id_vars="datetime", var_name="series_id", value_name="value")
series = series.groupby("series_id").apply(
    lambda x: x.set_index("datetime").asfreq("D"), include_groups=False
)
display(series)

exog = pd.DataFrame(
    {
        "exog_1": np.random.randint(1, 10, n),
        "exog_2": np.random.randint(1, 10, n),
        "exog_3": np.random.randint(1, 10, n),
        "exog_4": np.random.randint(1, 10, n),
        "exog_5": np.random.randint(1, 10, n),
        "exog_6": np.random.randint(1, 10, n),
    },
    index=pd.date_range(start="2023-01-01", periods=n, freq="D"),
)
exog.index.name= "datetime"
exog = [exog.assign(series_id=f"series_{i}") for i in range(1, n_series)]
# exog = [exog_i.copy().sample(frac=0.8).sort_index() for exog_i in exog]
exog = pd.concat(exog)
exog = exog.set_index(["series_id", exog.index])
display(exog)

# Formato dict
series_dict = {
    sid: series.loc[sid]['value'] for sid in series.index.levels[0]
}

exog_dict = {
    sid: exog.loc[sid]for sid in exog.index.levels[0]
}


value
series_id  datetime         
series_1   2023-01-01     55
           2023-01-02     70
           2023-01-03     96
           2023-01-04     57
           2023-01-05     49
...                      ...
series_999 2036-09-04     35
           2036-09-05      9
           2036-09-06     86
           2036-09-07     51
           2036-09-08      5

[4995000 rows x 1 columns]

exog_1  exog_2  exog_3  exog_4  exog_5  exog_6
series_id  datetime                                                  
series_1   2023-01-01       3       9       1       5       5       9
           2023-01-02       8       9       9       4       3       3
           2023-01-03       6       5       8       3       1       9
           2023-01-04       1       3       8       9       3       2
           2023-01-05       8       5       9       3       6       9
...                       ...     ...     ...     ...     ...     ...
series_999 2036-09-04       1       3       1       1       8       6
           2036-09-05       9       3       6       7       9       3
           2036-09-06       3       1       1       5       5       3
           2036-09-07       4       1       7       3       6       9
           2036-09-08       5       3       2       9       8       2

[4995000 rows x 6 columns]

In [82]:
%%timeit

l = []
for v in series_dict.values():
    l.append(v.index.freq)

343 μs ± 18.3 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [84]:
%%timeit

l = set()
series_names_in_ = series.index.levels[0].to_list()
for series_id in series_names_in_:
    l.add(series.loc[series_id].index.freq)

221 ms ± 24.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [85]:
%%timeit

l = set()
series_names_in_ = series.index.levels[0].to_list()
for series_id in series_names_in_:
    l.add(series.xs(series_id).index.freq)

205 ms ± 6.05 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [86]:
221000/343

644.3148688046647

In [59]:
%%timeit

series.xs('series_1')

242 μs ± 30.7 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [60]:
%%timeit

series.loc['series_1']

209 μs ± 15.3 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [ ]:
%%timeit

l = []
for v in series_dict.values():
    l.append(v.index.freq)

TimedeltaIndex([NaT, '1 days', '-4999 days'], dtype='timedelta64[ns]', name='datetime', freq=None)

In [15]:
freq_por_serie = series.groupby(level='series_id').apply(
    lambda x: x.index.get_level_values('datetime').freqstr
)
freq_por_serie

""


In [67]:
n = 5000
series = pd.DataFrame(
    {f"series_{i}": np.random.randint(1, 100, n) for i in range(1, 100)
    },
    index=pd.date_range(start="2023-01-01", periods=n, freq="D"),
)

freq = series.index.freq
series.index.name = "datetime"
series = series.reset_index()
series = pd.melt(series, id_vars="datetime", var_name="series_id", value_name="value")
series = series.groupby("series_id").apply(
    lambda x: x.set_index("datetime").asfreq("D"), include_groups=False
)

# Delete one random row
# sample_idx = series.sample(1).index
# print(sample_idx)
# series = series.drop(sample_idx)
# series

In [68]:
series_train = series.loc[series.index.get_level_values('datetime') < '2023-01-20']

In [71]:
series_train.loc['series_1'].index

DatetimeIndex(['2023-01-01', '2023-01-02', '2023-01-03', '2023-01-04',
               '2023-01-05', '2023-01-06', '2023-01-07', '2023-01-08',
               '2023-01-09', '2023-01-10', '2023-01-11', '2023-01-12',
               '2023-01-13', '2023-01-14', '2023-01-15', '2023-01-16',
               '2023-01-17', '2023-01-18', '2023-01-19'],
              dtype='datetime64[ns]', name='datetime', freq='D')

In [47]:
resultados = []

for sid, group in series:
    if int(sid[-1]) % 2 == 0:
        group = group.set_index("datetime").asfreq("D")
    else:
        group = group.set_index("datetime").asfreq("ME")
    group['series_id'] = sid  # Mantener columna identificadora si se pierde
    resultados.append(group)

series_final = pd.concat(resultados)
series_final.query('series_id == "series_2"').index

DatetimeIndex(['2023-01-01', '2023-01-02', '2023-01-03', '2023-01-04',
               '2023-01-05', '2023-01-06', '2023-01-07', '2023-01-08',
               '2023-01-09', '2023-01-10',
               ...
               '2036-08-30', '2036-08-31', '2036-09-01', '2036-09-02',
               '2036-09-03', '2036-09-04', '2036-09-05', '2036-09-06',
               '2036-09-07', '2036-09-08'],
              dtype='datetime64[ns]', name='datetime', length=5000, freq=None)

In [37]:
option_1 = series.index.levels[1] 
option_2 = pd.date_range(start=series.index.get_level_values('datetime').min(),
                        end=series.index.get_level_values('datetime').max(),
                        freq=None)

assert option_1.equals(option_2), "Los índices de las series no son iguales a la fecha de inicio y fin de las series."

In [48]:
import pandas as pd
import numpy as np

# Dos series con distinta frecuencia
df1 = pd.DataFrame({
    'series_id': 'A',
    'datetime': pd.date_range('2023-01-01', periods=5, freq='D'),
    'value': np.random.randn(5)
})

df2 = pd.DataFrame({
    'series_id': 'B',
    'datetime': pd.date_range('2023-01-01', periods=5, freq='ME'),
    'value': np.random.randn(5)
})

df = pd.concat([df1, df2])
df = df.set_index(['series_id', 'datetime'])

# Obtener la frecuencia de cada grupo
for serie, grupo in df.groupby(level=0):
    freq = grupo.index.get_level_values('datetime').freqstr
    print(f"Serie {serie} frecuencia: {freq}")


Serie A frecuencia: None
Serie B frecuencia: None


In [20]:
series.index.get_level_values('datetime').diff().unique()

TimedeltaIndex([         NaT,    '28 days',    '31 days',    '30 days',
                   '29 days', '-4991 days',     '1 days', '-4969 days',
                '-4999 days', '-4961 days'],
               dtype='timedelta64[ns]', name='datetime', freq=None)

In [24]:
index_1 = series.index.get_level_values('datetime')
index_0 = series.index.get_level_values('series_id')

index_1.to_series().groupby(index_0).diff().unique()

<TimedeltaArray>
[NaT, '28 days', '31 days', '30 days', '29 days', '1 days']
Length: 6, dtype: timedelta64[ns]

In [10]:
forecaster = ForecasterRecursiveMultiSeries(regressor=LGBMRegressor(), lags=5, transformer_exog=StandardScaler())
forecaster.transformer_series_ = {k: StandardScaler() for k in series.index.get_level_values(0).unique()}
forecaster.differentiator_ = {k: None for k in series.index.get_level_values(0).unique()}

In [11]:
import warnings
from skforecast.exceptions import MissingValuesWarning

warnings.simplefilter('ignore', category=MissingValuesWarning)  

In [12]:
# %%timeit -r 5 -n 5
# _ = forecaster._create_train_X_y(
#     series = series,
#     exog=exog
# )

In [13]:
%%timeit -r 5 -n 5
_ = forecaster._create_train_X_y_v2(
    series = series,
    exog=exog
)

319 ms ± 9.64 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)


In [20]:
_ = forecaster._create_train_X_y_v2(
                            series = series,
                            exog=exog
                        )

value


KeyError: 'value'

In [24]:
series_level_0 = series.index.get_level_values(0)
unique_series = series_level_0.unique()

# Opcional: crea un buffer de posiciones de inicio/fin para cada serie
idx_start = series_level_0.searchsorted(unique_series, side='left')
idx_start

array([     0,   5000,  10000,  15000,  20000,  25000,  30000,  35000,
        40000,  45000,  50000,  55000,  60000,  65000,  70000,  75000,
        80000,  85000,  90000,  95000, 100000, 105000, 110000, 115000,
       120000, 125000, 130000, 135000, 140000, 145000, 150000, 155000,
       160000, 165000, 170000, 175000, 180000, 185000, 190000, 195000,
       200000, 205000, 210000, 215000, 220000, 225000, 230000, 235000,
       240000, 245000, 250000, 255000, 260000, 265000, 270000, 275000,
       280000, 285000, 290000, 295000, 300000, 305000, 310000, 315000,
       320000, 325000, 330000, 335000, 340000, 345000, 350000, 355000,
       360000, 365000, 370000, 375000, 380000, 385000, 390000, 395000,
       400000, 405000, 410000, 415000, 420000, 425000, 430000, 435000,
       440000, 445000, 450000, 455000, 460000, 465000, 470000, 475000,
       480000, 485000, 490000])

In [14]:
def funt_to_profile(forecaster, series, exog):
    _ = forecaster._create_train_X_y_v2(
                            series = series,
                            exog=exog
                        )

%lprun -f forecaster._create_train_X_y_v2 funt_to_profile(forecaster, series, exog)

Timer unit: 1e-07 s

Total time: 0.66638 s
File: c:\Users\jaesc2\GitHub\skforecast\skforecast\recursive\_forecaster_recursive_multiseries.py
Function: _create_train_X_y_v2 at line 1272

Line #      Hits         Time  Per Hit   % Time  Line Contents
  1272                                               def _create_train_X_y_v2(
  1273                                                   self,
  1274                                                   series: pd.DataFrame,
  1275                                                   exog: pd.Series | pd.DataFrame | None = None,
  1276                                                   store_last_window: bool | list[str] = True,
  1277                                               ) -> tuple[
  1278                                                   pd.DataFrame,
  1279                                                   pd.Series,
  1280                                                   dict[str, pd.Index],
  1281                                      

In [16]:
%%timeit -r 5 -n 5
_ = forecaster._create_train_X_y_v3(
    series = series,
    exog=exog
)

493 ms ± 46.1 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)


In [ ]:
def funt_to_profile(forecaster, series, exog):
    _ = forecaster._create_train_X_y_v3(
                            series = series,
                            exog=exog
                        )

%lprun -f forecaster._create_train_X_y_v2 funt_to_profile(forecaster, series, exog)

In [20]:
%%timeit -r 5 -n 5
{
    sid: series.loc[sid]['value'] for sid in series.index.levels[0]
}

{
    sid: exog.loc[sid]for sid in exog.index.levels[0]
}

69.4 ms ± 5.9 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)


In [21]:
%%timeit -r 5 -n 5
_ = forecaster._create_train_X_y(
    series = series_dict,
    exog=exog_dict
)

413 ms ± 44.8 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)


In [22]:
def funt_to_profile(forecaster, series_dict, exog_dict):
    _ = forecaster._create_train_X_y(
                            series = series_dict,
                            exog=exog_dict
                        )

%lprun -f forecaster._create_train_X_y funt_to_profile(forecaster, series_dict, exog_dict)

Timer unit: 1e-07 s

Total time: 0.701712 s
File: c:\Users\jaesc2\Miniconda3\envs\skforecast_py11_2\Lib\site-packages\skforecast\recursive\_forecaster_recursive_multiseries.py
Function: _create_train_X_y at line 1087

Line #      Hits         Time  Per Hit   % Time  Line Contents
  1087                                               def _create_train_X_y(
  1088                                                   self,
  1089                                                   series: pd.DataFrame | dict[str, pd.Series | pd.DataFrame],
  1090                                                   exog: pd.Series | pd.DataFrame | dict[str, pd.Series | pd.DataFrame] | None = None,
  1091                                                   store_last_window: bool | list[str] = True,
  1092                                               ) -> tuple[
  1093                                                   pd.DataFrame,
  1094                                                   pd.Series,
  1095           

In [4]:
def funt_to_profile(series):
    check_preprocess_series(series=series)

%lprun -f check_preprocess_series funt_to_profile(series)

Timer unit: 1e-07 s

Total time: 0.064929 s
File: c:\Users\jaesc2\GitHub\skforecast\skforecast\utils\utils.py
Function: check_preprocess_series at line 2448

Line #      Hits         Time  Per Hit   % Time  Line Contents
  2448                                           def check_preprocess_series(
  2449                                               series: pd.DataFrame,
  2450                                           ) -> tuple[pd.DataFrame]:
  2451                                               """
  2452                                               Check and preprocess `series` argument in `ForecasterRecursiveMultiSeries`
  2453                                               class.
  2454                                           
  2455                                               Check if the indexes of the series are valid. If the temporal index of the
  2456                                               series is not a pandas DatetimeIndex index with frequency, it is overwritte

In [7]:
def funt_to_profile(series_dict):
    check_preprocess_series(series=series_dict)

%lprun -f check_preprocess_series funt_to_profile(series_dict)

Timer unit: 1e-07 s

Total time: 0.0202911 s
File: c:\Users\jaesc2\Miniconda3\envs\skforecast_py11_2\Lib\site-packages\skforecast\utils\utils.py
Function: check_preprocess_series at line 2448

Line #      Hits         Time  Per Hit   % Time  Line Contents
  2448                                           def check_preprocess_series(
  2449                                               series: pd.DataFrame | dict[str, pd.Series | pd.DataFrame],
  2450                                           ) -> tuple[dict[str, pd.Series], dict[str, pd.Index]]:
  2451                                               """
  2452                                               Check and preprocess `series` argument in `ForecasterRecursiveMultiSeries` class.
  2453                                           
  2454                                               - If `series` is a pandas DataFrame, it is converted to a dict of pandas 
  2455                                               Series and index is overwri

In [ ]:
# %%pyinstrument
# x_train, y_train = forecaster.create_train_X_y(
#     series = series_dict,
#     exog=exog_dict
# )

In [ ]:
import numpy as np
import pandas as pd
from numba import njit
from collections import defaultdict


n = 5000
series = pd.DataFrame(
    {i: np.random.randint(1, 100, n) for i in range(1, 100)
    },
    index=pd.date_range(start="2023-01-01", periods=n, freq="h"),
)

freq = series.index.freq
series.index.name = "datetime"
series = series.reset_index()
series = pd.melt(series, id_vars="datetime", var_name="series_id", value_name="value")
series = series.groupby("series_id").apply(
    lambda x: x.set_index("datetime").asfreq("D"), include_groups=False
)

In [ ]:


# Extract MultiIndex levels
ids = series.index.get_level_values(0).to_numpy()
dates = series.index.get_level_values(1).astype('datetime64[ns]').to_numpy()

# Ensure sorting by (id, datetime)
sort_idx = np.lexsort((dates, ids))
ids = ids[sort_idx]
dates_ns = dates[sort_idx].view('int64')

@njit
def most_common_deltas(ids, dates_ns, max_check=1000):
    result_ids = []
    result_deltas = []

    start = 0
    n = len(ids)

    while start < n:
        current_id = ids[start]
        end = start + 1
        while end < n and ids[end] == current_id:
            end += 1

        # Extract time series slice
        series = dates_ns[start:end]
        count = min(len(series) - 1, max_check)
        
        if count < 1:
            result_ids.append(current_id)
            result_deltas.append(-1)  # No data
            start = end
            continue

        delta_counts = {}
        for i in range(count):
            delta = series[i + 1] - series[i]
            if delta in delta_counts:
                delta_counts[delta] += 1
            else:
                delta_counts[delta] = 1

        # Find most common delta
        max_delta = -1
        max_count = -1
        for delta, cnt in delta_counts.items():
            if cnt > max_count:
                max_delta = delta
                max_count = cnt

        result_ids.append(current_id)
        result_deltas.append(max_delta)
        start = end

    return result_ids, result_deltas



In [ ]:

unique_ids, deltas_ns = most_common_deltas(ids, dates_ns)

# Convert to pandas timedelta64
freqs = {}
for uid, delta in zip(unique_ids, deltas_ns):
    if delta == -1:
        freqs[uid] = None  # Not enough data
    else:
        freqs[uid] = pd.Timedelta(delta, unit='ns')  # Convert to pandas Timedelta
freqs


In [13]:
freqstr_ = pd.date_range(start="2023-01-01", periods=10, freq='ME').freqstr

In [14]:
pd.date_range(start="2023-01-01", end="2023-01-10", freq=freqstr_)

DatetimeIndex([], dtype='datetime64[ns]', freq='ME')

In [12]:
s = pd.Series(np.arange(10), index=pd.date_range(start="2023-01-01", periods=10, freq='D'))
s

2023-01-01    0
2023-01-02    1
2023-01-03    2
2023-01-04    3
2023-01-05    4
2023-01-06    5
2023-01-07    6
2023-01-08    7
2023-01-09    8
2023-01-10    9
Freq: D, dtype: int64

In [13]:
s.loc['2021-01-01':'2024-01-05']

2023-01-01    0
2023-01-02    1
2023-01-03    2
2023-01-04    3
2023-01-05    4
2023-01-06    5
2023-01-07    6
2023-01-08    7
2023-01-09    8
2023-01-10    9
Freq: D, dtype: int64